In [3]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm

# %matplotlib inline

## Data Description
1. Trainning Set
    - 12K samples of house posted at May, June, July of 2018 at StreetEasy
2. Test Set 1
    - Samples of house posted at August 2018 with observed rent
    - 2000 Samples
3. Test Set 2 and Test Set 3
    - Samples of house posted at August 2018 without observed rent
    - 2000 Samples

In [4]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_set1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
test_set2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
# test_set3_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)


In [6]:
train_df.head(2).transpose()

rental_id,7236931,7331920
addr_unit,#22A,#406
building_id,551248,3373
bedrooms,4,0
bathrooms,3,1
size_sqft,1300,475
created_at,2018-06-04 16:31:06,2018-07-03 20:11:43
addr_street,645 OCEAN AVENUE,93 WORTH STREET
addr_city,Brooklyn,New York
addr_zip,11226,10013
addr_lat,40.647,40.7166


In [7]:
train_df.describe()

,building_id,bedrooms,bathrooms,size_sqft,addr_zip,addr_lat,addr_lon,bin,bbl,floor_count,...,has_washer_dryer,has_garage,has_roofdeck,has_concierge,has_pool,has_garden,has_childrens_playroom,rent,no_fee,floornumber
count,1.200000e+04,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,1.199900e+04,1.200000e+04,12000.000000,...,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,10723.000000
mean,1.529133e+06,1.657667,1.239500,850.465750,10682.779667,40.727507,-73.956837,2.370093e+06,2.287120e+09,10.266608,...,0.271667,0.170000,0.264083,0.208583,0.073167,0.132083,0.090500,3605.762917,0.490000,6.182738
std,4.301123e+06,1.079771,0.560208,515.773101,593.370964,0.053241,0.043024,1.242919e+06,1.202105e+09,12.019787,...,0.444837,0.375648,0.440862,0.406313,0.260421,0.338595,0.286909,2897.069446,0.499921,7.671545
min,7.300000e+01,0.000000,0.000000,0.000000,10001.000000,40.573898,-74.141570,1.000000e+06,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1250.000000,0.000000,0.000000
25%,6.209200e+04,1.000000,1.000000,610.000000,10023.000000,40.693256,-73.986076,1.052139e+06,1.011220e+09,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2250.000000,0.000000,2.000000
50%,2.495815e+05,2.000000,1.000000,800.000000,11103.000000,40.729028,-73.961700,3.019358e+06,3.006580e+09,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2900.000000,0.000000,3.000000
75%,8.225132e+05,2.000000,1.000000,1000.000000,11218.000000,40.764599,-73.936663,3.326098e+06,3.050580e+09,12.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3900.000000,1.000000,6.000000
max,1.877210e+07,8.000000,20.000000,10000.000000,11694.000000,40.909842,-73.730550,5.158986e+06,5.010640e+09,90.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,50000.000000,1.000000,78.000000


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 7236931 to 7422694
Data columns (total 38 columns):
addr_unit                 11909 non-null object
building_id               12000 non-null int64
bedrooms                  12000 non-null int64
bathrooms                 12000 non-null float64
size_sqft                 12000 non-null float64
created_at                12000 non-null object
addr_street               12000 non-null object
addr_city                 12000 non-null object
addr_zip                  12000 non-null int64
addr_lat                  12000 non-null float64
addr_lon                  12000 non-null float64
bin                       11999 non-null float64
bbl                       12000 non-null int64
floor_count               12000 non-null float64
year_built                11597 non-null float64
min_to_subway             11874 non-null float64
has_doorman               12000 non-null int64
has_elevator              12000 non-null int64
has_fireplace    

In [7]:
test_set1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 7499437 to 7463404
Data columns (total 38 columns):
addr_unit                 1981 non-null object
building_id               2000 non-null int64
bedrooms                  2000 non-null int64
bathrooms                 2000 non-null float64
size_sqft                 2000 non-null float64
created_at                2000 non-null object
addr_street               2000 non-null object
addr_city                 2000 non-null object
addr_zip                  2000 non-null int64
addr_lat                  2000 non-null float64
addr_lon                  2000 non-null float64
bin                       2000 non-null float64
bbl                       2000 non-null int64
floor_count               2000 non-null float64
year_built                1933 non-null float64
min_to_subway             1983 non-null float64
has_doorman               2000 non-null int64
has_elevator              2000 non-null int64
has_fireplace             2000 non-n

In [8]:
test_set2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 7428577 to 7443274
Data columns (total 38 columns):
addr_unit                 1971 non-null object
building_id               2000 non-null int64
bedrooms                  2000 non-null int64
bathrooms                 2000 non-null float64
size_sqft                 2000 non-null float64
created_at                2000 non-null object
addr_street               2000 non-null object
addr_city                 2000 non-null object
addr_zip                  2000 non-null int64
addr_lat                  2000 non-null float64
addr_lon                  2000 non-null float64
bin                       2000 non-null float64
bbl                       2000 non-null int64
floor_count               2000 non-null float64
year_built                1918 non-null float64
min_to_subway             1984 non-null float64
has_doorman               2000 non-null int64
has_elevator              2000 non-null int64
has_fireplace             2000 non-n